# Large-Scale Population Synthesis on HPC Facilities 🚀

**Tutorial goal**

This tutorial will cover how to setup and run a large, multi-metallicity population run on a HPC with slurm.

We will dive deeper into the `population_params.ini` file and the `Population` class for multi-metallicity populations.

If you haven't done so yet, export the path POSYDON environment variables. For example:

In [1]:
%env PATH_TO_POSYDON=/Users/simone/Google Drive/github/POSYDON-public/
%env PATH_TO_POSYDON_DATA=/Volumes/T7/

env: PATH_TO_POSYDON=/Users/simone/Google Drive/github/POSYDON-public/
env: PATH_TO_POSYDON_DATA=/Volumes/T7/


## Creating the Initialization File for multi-metallicity runs

Let's copy the default population synthesis ini file to your working directory.
Make sure you're on a cluster to be able to run and submit the jobs.


In [7]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON

path_to_params = os.path.join(PATH_TO_POSYDON, "posydon/popsyn/population_params_default.ini")
shutil.copyfile(path_to_params, './population_params.ini')

'./population_params.ini'

Open the `population_params.ini` file and do the following edits to run a large model at 8 differet metallicities:

- set `metallicity = [2., 1., 0.45, 0.2, 0.1, 0.01, 0.001, 0.0001]`
- set `number_of_binaries = 100`

You might also want to make sure the `dump_rate` is set to 10 binaries. 
`dump_rate = 10`

## Setting-up the Population Synthesis Model 

POSYDON provides `setup-popsyn` that you can use to setup a (multi-)metallicity population run on a HPC facility.

This will split each metallicity into a separate slurm job-array and a dependent job which will automatically merge the output of the separate jobs.

Below are two examples, but you might require to adjust the inputs for your email, cluster and available partitions.

In [ ]:
# example for yggdrasil
setup-popsyn population_params.ini --job_array=10 --walltime=00:14:00 --partition=debug-cpu --email=max.briel@unige.ch --account=briel

In [ ]:
# example for quest cluster
setup-popsyn population_params.ini --job-array=10 --walltime=00:14:00

`setup-popsyn --help` should provide a complete list of possible input parameters.


**walltime and job_array number fine-tuning**

The above examples will setup the population run with an array of 10 jobs for each metallicity. As such, each job will run 10 binaries of the 100 binaries per metallicity.

Fine-tuning the `dump_rate` compared to the number of binaries each job runs can be helpful. However, the larger the `dump_rate` the higher the memory footprint of each job.
As a default, 4Gb of RAM is requested per job.

Similarly, the `walltime` and `job_array` can be fine-tuned. A single binary takes about 1-2 seconds to run. Depending on the number of binaries each job does, you might want to raise or lower the walltime for optimal perfomance.

Instead of setting a `dump_rate`, it's also possible to set a `ram_per_cpu`. The code will try to stay below 90\% of this limit, but this is not always guaranteed due to additional python overhead.

After you've ran the above setup, you should now have several files in your work directory.

You can submit all job arrays and merged jobs using `slurm_submit.sh`


In [ ]:
sh slurm_submit.sh

If one of your runs fails, you can manually submit them again after fixing the issue.
The `*_logs` folder will contain the logs of each job in the job_array.

## Populations inspection

Once the runs and the mergers have finished. You should have 8 files named `MET_Zsun_population.h5`, where `MET` are the metallicities.

We will inspect two to check the number of binaries in the population.

In [8]:
from posydon.popsyn.synthetic_population import Population

file1 = '1e-01_Zsun_population.h5'
file2 = '1e-02_Zsun_population.h5'

pop1 = Population(file1)
print(pop1.mass_per_met)

pop2 = Population(file2)
print(pop2.mass_per_met)

FileNotFoundError: ``/Users/max/Documents/POSYDON/docs/_source/tutorials-examples/population-synthesis/1e-01_Zsun_population.h5`` does not exist

## Parsing the Population Synthesis Model Output

If everything is set up correctly, the simulation will generate 8 different population synthesis models, one for each metallicity containig 1 million binaries each. The simulation will take a few hours to complete. For convinience, we have already run the simulation and the results are available in the `.../POSYDON_data/tutorials/population-synthesis/example/` folder.

In [11]:
import os
from posydon.config import PATH_TO_POSYDON_DATA

path = os.path.join(PATH_TO_POSYDON_DATA, "POSYDON_data/tutorials/population-synthesis/example/")
files = sorted([f for f in os.listdir(path) if f.endswith('Zsun_population.h5')])
path_to_data = [os.path.join(path, file) for file in files] 
path_to_data

['/Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e+00_Zsun_population.h5',
 '/Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e-01_Zsun_population.h5',
 '/Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e-02_Zsun_population.h5',
 '/Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e-03_Zsun_population.h5',
 '/Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e-04_Zsun_population.h5',
 '/Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/2.00e+00_Zsun_population.h5',
 '/Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/2.00e-01_Zsun_population.h5',
 '/Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/4.50e-01_Zsun_population.h5']

Here we show how you can parse the simulation results and save the subpopulation of merging binary black holes (BBH).

In [12]:
from posydon.popsyn.synthetic_population import SyntheticPopulation

pop = SyntheticPopulation(path_to_ini='./population_params.ini', verbose=True)

pop.parse(path_to_data=path_to_data, S1_state='BH', S2_state='BH', binary_state='contact', invert_S1S2=False)
pop.save_pop(os.path.join(path,'BBH_population.h5'))

pop.df.head(10)

Binary count with (S1_state, S2_state, binary_state, binary_event) equal
to (BH, BH, contact, None)
in /Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e+00_Zsun_population.h5 are 233
in /Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e-01_Zsun_population.h5 are 2643
in /Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e-02_Zsun_population.h5 are 5974
in /Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e-03_Zsun_population.h5 are 8320
in /Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/1.00e-04_Zsun_population.h5 are 9683
in /Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/2.00e+00_Zsun_population.h5 are 121
in /Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/2.00e-01_Zsun_population.h5 are 3021
in /Volumes/T7/POSYDON_data/tutorials/population-synthesis/example/4.50e-01_Zsun_population.h5 are 761
Total binaries found are 30756


/Users/simone/Google Drive/github/POSYDON-public/posydon/popsyn/synthetic_population.py:300: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['state', 'event', 'step_names', 'S1_state', 'S2_state'], dtype='object')]

  self.df.to_hdf(path, key='history')
/Users/simone/Google Drive/github/POSYDON-public/posydon/popsyn/synthetic_population.py:301: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['state_i', 'event_i', 'step_names_i', 'state_f', 'event_f',
       'step_names_f', 'S1_state_i', 'S1_state_f', 'S1_SN_type', 'S2_state_i',
       'S2_state_f', 'S2_SN_type', 'interp_class_HMS_HMS',
       'interp_class_CO_HMS_RLO', 'interp_class_CO_HeMS',
       'interp_class_CO_HeMS_RLO', 'mt_history_HMS_HMS',
       'mt_history_CO_HM

Population successfully saved!


,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin,metallicity,simulated_mass_for_met,underlying_mass_for_met
binary_index,,,,,,,,,,,,,,,,,,,,,
3587,detached,ZAMS,0.000000e+00,2.493602e+01,0.000000,NaN,initial_cond,0.000000,H-rich_Core_H_burning,70.069756,...,NaN,7.155000e-01,2.703000e-01,NaN,NaN,NaN,NaN,0.0142,2.913438e+07,1.447893e+08
3587,contact,oDoubleCE1,3.631450e+06,6.304073e+01,0.000000,-2.989131,step_HMS_HMS,0.037464,H-rich_Core_He_burning,44.118926,...,0.000000,0.000000e+00,9.828315e-01,4.246537e-01,0.561493,0.577444,0.760854,0.0142,2.913438e+07,1.447893e+08
3587,detached,NaN,3.631450e+06,2.371597e-01,0.000000,NaN,step_CE,0.000137,stripped_He_Core_He_burning,35.566786,...,0.000000,0.000000e+00,9.828315e-01,1.000000e-02,0.975800,NaN,NaN,0.0142,2.913438e+07,1.447893e+08
3587,detached,CC1,4.007490e+06,1.226986e+00,0.000000,NaN,step_detached,0.777758,stripped_He_Central_C_depletion,13.620462,...,0.401483,1.917729e-34,1.605147e-02,9.893273e-100,0.247607,0.006843,0.077976,0.0142,2.913438e+07,1.447893e+08
3587,detached,NaN,4.007490e+06,1.358968e+00,0.101109,NaN,step_SN,0.152370,BH,13.120462,...,0.401483,1.917729e-34,1.605147e-02,9.893273e-100,0.247607,0.006843,0.077976,0.0142,2.913438e+07,1.447893e+08
3587,detached,redirect,4.007490e+06,1.358968e+00,0.101109,NaN,step_CO_HeMS,0.000102,BH,13.120462,...,0.401483,1.917729e-34,1.605147e-02,9.893273e-100,0.247607,0.006843,0.077976,0.0142,2.913438e+07,1.447893e+08
3587,detached,CC2,4.027170e+06,1.377894e+00,0.101108,NaN,step_detached,0.452186,BH,13.120462,...,0.130995,0.000000e+00,7.706932e-13,1.000000e-99,0.226684,0.015362,0.060340,0.0142,2.913438e+07,1.447893e+08
3587,detached,NaN,4.027170e+06,1.611464e+00,0.048133,NaN,step_SN,0.149304,BH,13.120462,...,NaN,NaN,NaN,NaN,NaN,NaN,0.064849,0.0142,2.913438e+07,1.447893e+08
3587,contact,CO_contact,2.917891e+09,2.638756e-08,0.000000,NaN,step_dco,1.252216,BH,13.120462,...,NaN,NaN,NaN,NaN,NaN,NaN,0.064849,0.0142,2.913438e+07,1.447893e+08


You can now do the same for the other subpopulations of interest. Try sorting black hole-neutron star systems (BHNS; remember to set `invert_S1S2 = True` to find BHNS systems where the NS is formed first) and binary neutron star systems (BNS).